In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import inchi
from tqdm import tqdm
from time import sleep
from tqdm.notebook import tqdm

from chembl_structure_pipeline import standardizer as ChEMBL_standardizer
from papyrus_structure_pipeline import standardize

[ForwardRef('ExtensionArray'), <class 'numpy.ndarray'>]
[ForwardRef('ExtensionArray'), <class 'numpy.ndarray'>, ForwardRef('Index'), ForwardRef('Series')]
[<class 'str'>, <class 'float'>, <class 'bool'>]
[ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta')]
[ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta'), ForwardRef('Interval')]
[<class 'str'>, <class 'float'>, <class 'bool'>, ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta'), ForwardRef('Interval'), <class 'numpy.datetime64'>, <class 'numpy.timedelta64'>, <class 'datetime.datetime'>]
[ForwardRef('Timestamp'), <class 'datetime.datetime'>, <class 'numpy.datetime64'>, <class 'numpy.int64'>, <class 'float'>, <class 'str'>]
[ForwardRef('Timedelta'), <class 'datetime.timedelta'>, <class 'numpy.timedelta64'>, <class 'numpy.int64'>, <class 'float'>, <class 'str'>]
[<class 'str'>, <class 'datetime.tzinfo'>]
[<class 'str'>, <class 'int'>]
[typing.Hashable, typing.Sequence[typing.Ha

[14:18:40] Initializing Normalizer


In [2]:
#Read the BBB data: binary (train) and continuous "log" (validation)

bi_bbb = pd.read_excel('raw_data/bbb/bbb_permeability.xls')
log_bbb = pd.read_excel('raw_data/bbb/logbbb.xls')

[typing.List[int], typing.List[str]]


In [3]:
#Categorize log_bbb data as BBB permeable (LogBB ≥ -1) and BBB non-permeable (LogBB < -1)

def compare_values(row):
    if row['logBB'] < -1:
        return 'BBB-'
    elif row['logBB'] >= -1:
        return 'BBB+'

log_bbb['status_bbb'] = log_bbb.apply(compare_values, axis=1)


,SMILES,CASRN,EXTERNALID,N,NAME,ARTICLEID,PUBMEDID,PAGE,TABLE,logBB,...,"logBB {measured, converted}",UNIT {logBB}.1,Tissue,Dataset,Species,VOC,Drug_Class,DataSetName,comment (chemical),status_bbb
0,O=C1N2CCCC(O)C2=NC(C)=C1CCN3CCC(C4=NOC5=C4C=CC...,NaN,NaN,24,9-OH Risperidone,A23,10554091.0,1515.0,1,-0.67,...,21.38,%,Blood,NaN,NaN,NaN,NaN,NaN,NaN,BBB+
1,CN1CC2C3=CC=CC(C)=C3OC4=CC=CC=C4C2(CC1)O,NaN,NaN,5,Beloxepin (Org4428),A23,10554091.0,1515.0,1,0.82,...,660.70,%,Blood,NaN,NaN,NaN,NaN,NaN,NaN,BBB+
2,NC(N1C2=C(C=CC3=C1C=CC=C3)C=CC=C2)=O,NaN,NaN,14,carbamazepine,A23,10554091.0,1515.0,1,0.00,...,100.00,%,Blood,NaN,NaN,NaN,NaN,NaN,NaN,BBB+
3,ClC1=CC=C2N(C(NC2=C1)=O)C3CCN(CC3)CCCN4C(NC5=C...,NaN,NaN,21,domperidone,A23,10554091.0,1515.0,1,-0.78,...,16.60,%,Blood,NaN,NaN,NaN,NaN,NaN,NaN,BBB+
4,CN1CCN2C(C3=CC=CC=C3CC4=CC=CC=C24)C1,NaN,NaN,3,Mianserin,A23,10554091.0,1515.0,1,0.99,...,977.20,%,Blood,NaN,NaN,NaN,NaN,NaN,NaN,BBB+


In [4]:
#Keep only the needed column and rename them
bbb_val = log_bbb[['SMILES','logBB','status_bbb']]
bbb_train = bi_bbb[['SMILES','BBB permeability (qualitative)']]

bbb_val=bbb_val.rename(columns={'SMILES':'SMILES_raw'})
bbb_train = bbb_train.rename(columns={'SMILES':'SMILES_raw','BBB permeability (qualitative)':"status_bbb"})

In [5]:
#Remove rows without SMILES

def remove_nan_smiles(df):

    df = df[~(df['SMILES_raw'].isna())]
    df = df.reset_index(drop=True)

    return df

In [6]:
#Papyrus Standardization

def create_sd_smiles(sd_mol):
    try:
        standardized_smiles =  Chem.MolToSmiles(sd_mol)
        return standardized_smiles
    except Exception as e:
        print(f"An sd_smiles error occurred: {str(e)}")
        return None
    
#Create InChI keys from standardized molecules
def mol_to_inchi_key(sd_mol):
    if sd_mol is not None:

        inchi_str = inchi.MolToInchi(sd_mol)
        inchi_key = inchi.InchiToInchiKey(inchi_str)
    else:
        inchi_key = None   
    return inchi_key

def standardize_molecule(mol):
    standardized_mol =  standardize(mol,raise_error=False )
    return standardized_mol

#Standardize 

def standardize_workflow(df_raw):
    for i in range(0,len(df_raw)):
        smiles =df_raw.at[i,'SMILES_raw']
        mol = Chem.MolFromSmiles(smiles)
        sd_mol =  standardize_molecule(mol)
        sd_smiles = create_sd_smiles(sd_mol)
        sd_inchi_key = mol_to_inchi_key(sd_mol)
        df_raw.at[i,'papyrus_SMILES'] = sd_smiles
        df_raw.at[i,'papyrus_inchi_key'] = sd_inchi_key

    print(f'df length after standardization: {len(df_raw)}')

    return df_raw

In [7]:
#Check for missing inchi key

def missing_inchi(df_raw):
    smiles_nan = df_raw['papyrus_SMILES'].isna().sum()
    inchikey_nan =df_raw['papyrus_inchi_key'].isna().sum()
    print(f'DB length: {len(df_raw)},        SMILES nan: {smiles_nan},        inchi key nan: {inchikey_nan}')

    #Remove rows with missing inchikey
    df_valid_inchi= df_raw[((df_raw['papyrus_inchi_key'].notna()))]
    print('-----remove missing inchikey----')
    print(f'updated length: {len(df_valid_inchi)}')

    return df_valid_inchi

In [8]:
def inchi_first_part(inchi):
    return inchi.split('-')[0]

def create_connectivity_inchi(df):
    df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)
    
    return df

In [10]:
#Check for duplicates

def remove_duplicates(df):
    print(f'length: {len(df)}')
    inchi_un = df['inchi_connectivity'].nunique()
    print(f'unique_inchi: {inchi_un}')


    unique_counts = df.groupby('inchi_connectivity')['status_bbb'].nunique()
    duplicates_diff_class = unique_counts[unique_counts > 1].index

    print(f'Contradicting duplicates: {len(duplicates_diff_class)}')

    #Remove duplicates
    df = df[~(df['inchi_connectivity'].isin(duplicates_diff_class))]
    print(len(df))
    print(df['inchi_connectivity'].nunique())

    df=df.drop_duplicates(subset=['inchi_connectivity'], keep="first").reset_index(drop=True)
    #df['inchi_stereo'].values_counts

    return df

In [11]:
#Code classes

def class_code(df):
    df['status_bbb'] = df['status_bbb'].replace({'BBB+': 1, 'BBB-': 0})

    return df


In [13]:
#Prepare the training data and save

df = bbb_train
df = remove_nan_smiles(df)
df_sd = standardize_workflow(df)
df_valid = missing_inchi(df_sd)
df_connectivity_inchi = create_connectivity_inchi(df_valid)
df_no_duplicates = remove_duplicates(df_connectivity_inchi)
df_class_coded = class_code(df_no_duplicates)

df_class_coded.to_csv('train_data/kadar_bbb_train.csv')

[14:30:50] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

[14:30:50] WARNING: Omitted undefined stereo

[14:30:50] WARNING: Omitted undefined stereo

[14:30:50] WARNING: Omitted undefined stereo

[14:30:50] WARNING: Omitted undefined stereo

[14:30:50] WARNING: Omitted undefined stereo

[14:30:50] WARNING: Omitted undefined stereo

[14:30:50] WARNING: Omitted undefined stereo

[14:30:50] WARNING: Omitted undefined stereo

[14:30:50] WARNING: Omitted undefined stereo

[14:30:50] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefi

An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

[14:30:51] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefined stereo

[14:30:51] WARNING: Omitted undefined stereo

[14:30:52] WARNING: Omitted undefined stereo

[14:30:52] WARNING: Omitted undefined stereo

[14:30:52] WARNING: Omitted undefined stereo

[14:30:52] Explicit valence for atom # 18 N, 4, is greater than permitted


...

[14:39:43] WARNING: Omitted undefined stereo

[14:39:43] WARNING: Omitted undefined stereo

[14:39:43] WARNING: Omitted undefined stereo

[14:39:44] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

[14:39:44] WARNING: Omitted undefined stereo

[14:39:45] WARNING: Omitted undefined stereo



df length after standardization: 17097
DB length: 17097,        SMILES nan: 2506,        inchi key nan: 2506
-----remove missing inchikey----
updated length: 14591
length: 14591
unique_inchi: 2237
Contradicting duplicates: 196
12426
2041


[14:39:45] WARNING: Omitted undefined stereo

C:\Users\Lily\AppData\Local\Temp\ipykernel_7076\2591752290.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)


In [16]:
#Prepare the validation data and save
df = bbb_val

df = remove_nan_smiles(df)
df_sd = standardize_workflow(df)
df_valid = missing_inchi(df_sd)
df_connectivity_inchi = create_connectivity_inchi(df_valid)
df_no_duplicates = remove_duplicates(df_connectivity_inchi)
df_class_coded = class_code(df_no_duplicates)

df_class_coded.to_csv('val_data/kadar_bbb_val.csv')

[14:43:55] WARNING: Omitted undefined stereo

[14:43:55] WARNING: Omitted undefined stereo

[14:43:55] WARNING: Omitted undefined stereo

[14:43:55] WARNING: Omitted undefined stereo

[14:43:55] WARNING: Omitted undefined stereo

[14:43:55] WARNING: Omitted undefined stereo

[14:43:55] WARNING: Omitted undefined stereo

[14:43:55] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)


[14:43:56] WARNING: Omitted undefined stereo

[14:43:56] WARNING: Charges were rearranged

[14:43:56] WARNING: Charges were rearranged

[14:43:56] WARNING: Charges were rearranged

[14:43:56] WARNING: Charges were rearranged

[14:43:56] WARNING: Charges were rearranged

[14:43:56] WARNING: Charges were rearranged

[14:43:56] WARNING: Charges were rearranged



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)


[14:43:56] WARNING: Omitted undefined stereo

[14:43:56] WARNING: Omitted undefined stereo

[14:43:56] WARNING: Omitted undefined stereo

[14:43:56] WARNING: Charges were rearranged

[14:43:56] WARNING: Charges were rearranged

[14:43:56] WARNING: Charges were rearranged

[14:43:56] WARNING: Charges were rearranged

[14:43:56] WARNING: Charges were rearranged



...

[14:45:42] WARNING: Omitted undefined stereo

[14:45:42] WARNING: Omitted undefined stereo

[14:45:42] WARNING: Omitted undefined stereo

[14:45:42] WARNING: Omitted undefined stereo

[14:45:42] WARNING: Omitted undefined stereo

[14:45:42] WARNING: Omitted undefined stereo

[14:45:42] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)


[14:45:42] WARNING: Omitted undefined stereo

[14:45:42] WARNING: Omitted undefined stereo

[14:45:42] WARNING: Omitted undefined stereo

[14:45:43] WARNING: Omitted undefined stereo

[14:45:43] WARNING: Omitted undefined stereo

[14:45:43] WARNING: Omitted undefined stereo

[14:45:43] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

[14:45:43] WARNING: Omitted undefined stereo

[14:45:43] WARNING: Omitted undefined stereo

[14:45:43] WARNING: Omitted undefined stereo

[14:45:43] WARNING: Charges were rearranged; Omitted undefined stereo

C:\Users\Lily\AppData\Local\Temp\ipykernel_7076\2591752290.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)
